In [1]:
import requests
import json
import os

# Target URL
url = 'https://celestrak.org/NORAD/elements/gp.php?GROUP=last-30-days&FORMAT=json'

# Local save directory
out_dir = 'tle_json'
os.makedirs(out_dir, exist_ok=True)
out_path = os.path.join(out_dir, 'last_30_days.json')

# 1. Send request
resp = requests.get(url)
resp.raise_for_status()  # Raises an exception if the response status is not 200

# 2. Parse into a Python object (list/dict)
data = resp.json()

# 3. Write to local file
with open(out_path, 'w', encoding='utf-8') as f:
    # indent=2 for readability, ensure_ascii=False to retain Unicode characters
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f'Saved to {out_path}, total {len(data)} records')




Saved to tle_json\last_30_days.json, total 163 records


In [2]:
#!/usr/bin/env python3
# compute_state_vectors.py

import json
import numpy as np
import pandas as pd
from astropy import units as u
from astropy.time import Time
from poliastro.bodies import Earth
from poliastro.twobody.orbit import Orbit

# Input JSON file path
INPUT_JSON = 'tle_json/last_30_days.json'
# Output CSV file path
OUTPUT_CSV = 'last_30_days_state_vectors_corrected.csv'

def main():
    # 1. Load GP JSON data
    with open(INPUT_JSON, 'r', encoding='utf-8') as f:
        sats = json.load(f)

    records = []

    for sat in sats:
        name = sat.get('OBJECT_NAME', 'UNKNOWN')
        epoch_str = sat['EPOCH']  # ISO-format timestamp
        epoch = Time(epoch_str, format='isot', scale='utc')

        # 2. Classical orbital elements (from the GP JSON fields)
        inc  = sat['INCLINATION']        * u.deg
        raan = sat['RA_OF_ASC_NODE']     * u.deg
        ecc  = sat['ECCENTRICITY']       * u.one
        argp = sat['ARG_OF_PERICENTER']  * u.deg
        M    = sat['MEAN_ANOMALY']       * u.deg

        # 3. Convert mean motion from rev/day → rad/s
        #    sat['MEAN_MOTION'] is given in rev/day
        mean_motion_rev_per_day = sat['MEAN_MOTION'] * u.one / u.day
        # Convert to 1/s, then multiply by 2π to get rad/s
        n_rad_s = mean_motion_rev_per_day.to(1 / u.s) * (2 * np.pi)

        # 4. Compute semi-major axis a (km) from Kepler's third law
        mu = Earth.k.to(u.km**3 / u.s**2)
        a = (mu / n_rad_s**2) ** (1/3)

        # 5. Solve for eccentric anomaly E, then compute true anomaly ν
        M_rad = M.to(u.rad).value
        e = ecc.value
        E = M_rad  # Initial guess
        for _ in range(100):
            E = E - (E - e * np.sin(E) - M_rad) / (1 - e * np.cos(E))
        nu = 2 * np.arctan2(
            np.sqrt(1 + e) * np.sin(E / 2),
            np.sqrt(1 - e) * np.cos(E / 2)
        ) * u.rad

        # 6. Build the Orbit and extract the state vector
        orb = Orbit.from_classical(
            attractor=Earth,
            a=a,
            ecc=ecc,
            inc=inc,
            raan=raan,
            argp=argp,
            nu=nu,
            epoch=epoch
        )
        r_vec = orb.r.to(u.km).value     # [x, y, z] in km
        v_vec = orb.v.to(u.km / u.s).value  # [vx, vy, vz] in km/s

        records.append({
            'OBJECT_NAME': name,
            'EPOCH':       epoch_str,
            'x_km':        r_vec[0],
            'y_km':        r_vec[1],
            'z_km':        r_vec[2],
            'vx_km_s':     v_vec[0],
            'vy_km_s':     v_vec[1],
            'vz_km_s':     v_vec[2],
        })

    # 7. Save results
    df = pd.DataFrame(records)
    df.to_csv(OUTPUT_CSV, index=False)
    print(f'✅ Generated {len(df)} state vectors and saved to {OUTPUT_CSV}')

if __name__ == '__main__':
    main()

# If you need to inspect the first few rows after running:
# import pandas as pd
# df = pd.read_csv(OUTPUT_CSV)
# print(df.head())



✅ Generated 163 state vectors and saved to last_30_days_state_vectors_corrected.csv


In [3]:
import pandas as pd
import numpy as np

# Input CSV: contains state vectors x, y, z, vx, vy, vz
input_csv = 'last_30_days_state_vectors_corrected.csv'
# Output CSV: will include additional columns v_r_km_s and v_t_km_s
output_csv = 'state_vectors_rt.csv'

# 1. Read the data
df = pd.read_csv(input_csv)

# 2. Compute radial and transverse velocities
def compute_rt_velocities(row):
    # Position vector
    r = np.array([row['x_km'], row['y_km'], row['z_km']])
    # Velocity vector
    v = np.array([row['vx_km_s'], row['vy_km_s'], row['vz_km_s']])
    # Unit radial vector
    r_hat = r / np.linalg.norm(r)
    # Radial velocity = v · r_hat
    v_r = np.dot(v, r_hat)
    # Transverse velocity vector = v - v_r * r_hat
    v_t_vec = v - v_r * r_hat
    # Transverse speed magnitude
    v_t = np.linalg.norm(v_t_vec)
    return pd.Series({'v_r_km_s': v_r, 'v_t_km_s': v_t})

# Apply to each row and concatenate the new columns
rt = df.apply(compute_rt_velocities, axis=1)
df = pd.concat([df, rt], axis=1)

# 3. Save the results
df.to_csv(output_csv, index=False)

print(f'✅ Generated file with radial/transverse velocities: {output_csv}')



✅ Generated file with radial/transverse velocities: state_vectors_rt.csv
